In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from scipy.stats import linregress

# Import API key
from api_key import g_key

# Store Part I results into DataFrame
#Load the csv exported in Part I to a DataFrame

In [6]:
import csv

weather_data_csv_path = os.path.join("..", "output_data", "weather_data.csv")

weather_data = pd.read_csv(weather_data_csv_path)                                
    
weather_data.head(500)

,Unnamed: 0,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City_found
0,0,22.68,-3.98,102.87,10,0,14.81,ML,1597089302,Taoudenni
1,1,-29.66,17.89,45.37,59,0,3.42,ZA,1597089302,Springbok
2,2,35.16,112.86,84.00,1,97,4.97,CN,1597089303,Xixiang
3,3,60.34,102.28,58.03,68,0,1.52,RU,1597089303,Vanavara
4,4,-34.53,20.04,50.00,93,0,2.24,ZA,1597089303,Bredasdorp
...,...,...,...,...,...,...,...,...,...,...
495,495,-5.04,18.82,82.47,46,73,2.64,CD,1597089420,Kikwit
496,496,46.11,39.42,74.53,39,19,11.10,RU,1597089420,Novoplatnirovskaya
497,497,-20.01,57.66,64.00,71,2,5.01,MU,1597089420,Grand Gaube
498,498,59.97,23.43,61.00,78,0,7.47,FI,1597089420,Ekenäs


# Humidity Heatmap
Configure gmaps. Use the Lat and Lng as locations and Humidity as the weight. Add Heatmap layer to map.

In [7]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

locations = weather_data[["Latitude","Longitude"]]

Humidity = weather_data["Humidity"]

query_url = f"https://maps.googleapis.com/maps/api/geocode/json?" + "location" + "&location_type=ROOFTOP&result_type=street_address&g_key"
#url = "https://maps.googleapis.com/maps/api/geocode/json?latlng"

locations

,Latitude,Longitude
0,22.68,-3.98
1,-29.66,17.89
2,35.16,112.86
3,60.34,102.28
4,-34.53,20.04
...,...,...
569,57.05,-135.33
570,53.60,7.20
571,7.70,28.00
572,6.74,2.66


In [8]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity,
                                 dissipating = False, max_intensity=300,
                                 point_radius = 3)

fig.add_layer(heat_layer)
plt.savefig('../Images/HumidityHeatMap.png')
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

# Create new DataFrame fitting weather criteria
Narrow down the cities to fit weather conditions. Drop any rows will null values.

In [9]:
#Create a df for vacation weather where Max Temp is between 69-78 degrees F, Humidity == 0,
#Cloudinessbetween 2-6, Wind Speed between 4-6 mph

lat = []
lon = []
temp_max = []
humidity = []
cloudiness = []
wind_speed = []
country = []
date = []
city_found = []

vacation_dict = {
    "Latitude": lat,
    "Longitude": lon,
    "Max Temp": temp_max,
    "Humidity": humidity,
    "Cloudiness": cloudiness,
    "Wind Speed": wind_speed,
    "Country": country,
    "Date": date,
    "City_found": city_found,
 }

In [12]:
import csv

vacation_weather_csv_path = os.path.join("..", "output_data", "weather_data.csv")

vacation_weather = pd.read_csv(weather_data_csv_path)                                
    
vacation_weather.head(500)

,Unnamed: 0,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,City_found
0,0,22.68,-3.98,102.87,10,0,14.81,ML,1597089302,Taoudenni
1,1,-29.66,17.89,45.37,59,0,3.42,ZA,1597089302,Springbok
2,2,35.16,112.86,84.00,1,97,4.97,CN,1597089303,Xixiang
3,3,60.34,102.28,58.03,68,0,1.52,RU,1597089303,Vanavara
4,4,-34.53,20.04,50.00,93,0,2.24,ZA,1597089303,Bredasdorp
...,...,...,...,...,...,...,...,...,...,...
495,495,-5.04,18.82,82.47,46,73,2.64,CD,1597089420,Kikwit
496,496,46.11,39.42,74.53,39,19,11.10,RU,1597089420,Novoplatnirovskaya
497,497,-20.01,57.66,64.00,71,2,5.01,MU,1597089420,Grand Gaube
498,498,59.97,23.43,61.00,78,0,7.47,FI,1597089420,Ekenäs


In [23]:
# delete all rows with column "Max Temp" has value 69 to 78 
indexNames = vacation_weather[((vacation_weather["Max Temp"] >= 69) & (vacation_weather["Max Temp"] <= 78) &
                              (vacation_weather["Humidity"] == 0) & ((vacation_weather["Cloudiness"] >= 2) & (vacation_weather["Cloudiness"] <= 6)) &
                             (vacation_weather["Wind Speed"] >= 4) & (vacation_weather["Wind Speed"] <= 6))].indexNames.drop(indexNames,inplace=True)

# delete all rows with column "Humidity" has value greater than 0  
#indexNames = vacation_weather[(vacation_weather["Humidity"] == 0)].index
#vacation_weather.drop(indexNames , inplace=True)

# delete all rows with column "Cloudiness" has value 2 to 6
#indexNames = vacation_weather[ (vacation_weather["Cloudiness"] >= 2) & (vacation_weather["Cloudiness"] <= 6) ].index
#vacation_weather.drop(indexNames , inplace=True)

# delete all rows with column "Wind Speed" has value 6 to 10 
#indexNames = vacation_weather[ (vacation_weather["Wind Speed"] >= 4) & (vacation_weather["Wind Speed"] <= 6) ].index
#vacation_weather.drop(indexNames , inplace=True)


AttributeError: 'DataFrame' object has no attribute 'indexNames'

In [11]:
#save vacation_weather as csv file to output_data ("r../output_data/vacation_weather,csv")
#vacation_weather.to_csv(r'../output_data/vacation_weather.csv', index='false')


# Hotel Map
Store into variable named hotel_df. Add a "Hotel Name" column to the DataFrame. Set parameters to search for hotels with 5000 meters. Hit the Google Places API for each city's coordinates. Store the first Hotel result into the DataFrame. Plot markers on top of the heatmap.

In [ ]:
hotel = pd.DataFrame.loc

# NOTE: Do not change any of the code in this cell
Using the template add the hotel marks to the heatmap¶
info_box_template = """

Name
{Hotel Name}
City
{City}
Country
{Country}
""" # Store the DataFrame Row # NOTE: be sure to update with your DataFrame name hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()] locations = hotel_df[["Lat", "Lng"]]

In [ ]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Add marker layer ontop of heat map¶
Display figure

In [ ]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity,
                                 dissipating = False, max_intensity=300,
                                 point_radius = 5000)

fig.add_layer(heat_layer)
plt.savefig('../Images/HotelHeatMap.png')
fig